## Strategy
1. Basemodel with effnetB0
    - n_fold cross validation (done)
    - integrate weight and biases (done)
    - fp16 (done, swinTransformer was not able to learn with fp16)
    - cosine annealing scheduler (done)
    - gradcam (done, seperate notebook)
2. retry with swinTransfomer
3. Augmentation (done)
    - Random left right flip 
    - Random Crop resized 
    
    
<!-- 4. Blur detection
5. Dog cat detection
6. Multi-pet (group) detection
7. % coverage detection -->

In [ ]:
!pip install timm
!pip install wandb --upgrade

In [ ]:
# pytorch lib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import timm


# image agumentation lib
import albumentations as A
from albumentations.pytorch import ToTensorV2

# numeric lib
import numpy as np
import pandas as pd

# image library
from PIL import Image

#python lib
import random
import os
import tqdm

# ploting library
import matplotlib.pyplot as plt
import wandb 

#sklearn library
from sklearn.model_selection import StratifiedKFold

## Config File

In [ ]:
class Config:
    seed = 42
    epochs = 5
    train_img_dir = "../input/petfinder-pawpularity-score/train"
    image_size = 384
    n_splits = 5
    model_name = 'swin_large_patch4_window12_384' #'efficientnet_b0'
    train_batchsize = 8
    val_batchsize = 8
    debug = False
    
    fp16 = False
    
    #optimizer
    optimizer = "Adam"
    
    if optimizer == "Adam":
        optimizer_params = dict(
            lr = 1e-4,
            betas = (0.9, 0.999),
            eps = 1e-8,
            weight_decay = 0,
            amsgrad = False
        )
        
    elif optimizer == "SGD":
        optimizer_params = dict(
            lr = 1e-3,
            weight_decay = 0,
            dampening  = 0,
            nesterov = False
        ) 
        
    # Scheduler
    scheduler = 'CosineAnnealingLR' # CosineAnnealingLR, ReduceLROnPlateau
    if scheduler == "CosineAnnealingLR":
        scheduler_params = dict(
            T_max = epochs,
            eta_min = 0,
            last_epoch = -1,
            verbose = False
        )
    elif scheduler == "ReduceLROnPlateau":
        scheduler_params = dict(
            mode = "min",
            factor= 0.1,
            patience=4,
            threshold=1e-4,
            min_lr = 1e-6
        )

In [ ]:
## login to wandb and create project
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandkey = user_secrets.get_secret("wand_key")


def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

wandb.login(key=wandkey)
run = wandb.init(project="Pawpularity",
                 name=Config.model_name,
                 config = class2dict(Config),
                 group=Config.model_name)

## Seed everything

In [ ]:
def seed_python(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
def seed_torch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
seed_python(Config.seed)
seed_torch(Config.seed)

In [ ]:
class ContinuousStratifiedKFold(StratifiedKFold):
    def __init__(self, *args, **kwargs):
        super(ContinuousStratifiedKFold, self).__init__(*args, **kwargs)
    
    def split(self, X, y, groups=None):
        n_bins = int(np.floor(1+np.log2(len(y))))
        new_y = pd.cut(y, n_bins, labels=False)
        return super().split(X, new_y, groups)


## Loss function

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train_df['filepath'] = train_df['Id'].map(lambda x : os.path.join(Config.train_img_dir, x+".jpg"))
train_df['n_fold'] = -1

if Config.debug:
    train_df = train_df.sample(300)

    
train_df = train_df.reset_index()

kfold = ContinuousStratifiedKFold(n_splits=Config.n_splits, shuffle=True, random_state= Config.seed)

for fold_num, (train_idx, test_idx) in enumerate(kfold.split(train_df['Pawpularity'],train_df['Pawpularity'])):
    train_df.loc[test_idx, "n_fold"]= fold_num 

train_df.to_csv("training_kfold.csv")
train_df.head()

## Creating the dataset

In [ ]:
class PawDataset(Dataset):
    def __init__(self, image_fps, targets, transforms):
        super(PawDataset, self).__init__()
        self.image_fps = image_fps
        self.transforms = transforms
        self.targets = targets
        
    def __len__(self):
        return len(self.image_fps)
    
    def __getitem__(self, idx):
        image_fp = self.image_fps[idx]
        target = torch.tensor(self.targets[idx]).float()
        
        img = np.array(Image.open(image_fp))
        img = self.transforms(image=img)['image']
        return img, target

# Test
# transform = A.Compose([
#     A.Resize(Config.image_size, Config.image_size,p=1),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
#     ToTensorV2()
# ], p=1)

# test_dataset = PawDataset(["../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg"], [1], transform)
# plt.imshow(torch.permute(test_dataset[0][0], (1,2,0)).numpy())

## Creating the model

In [ ]:
class PawModel(nn.Module):
    def __init__(self, model_name):
        super(PawModel, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=True, in_chans=3) #this will create a model with classifier
        self.backbone.head = nn.Linear(self.backbone.head.in_features, 128)
#         self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(p=0.1)
        self.fcn = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        
    def forward(self,image_array):
        x = self.backbone(image_array)
        x = self.dropout(x)
        x = self.fcn(x)
        return self.output(x)
        
#Test
# model = PawModel()
# model(test_dataset[0][0].unsqueeze(0))

## Helper functions

In [ ]:
class AverageMeter():
    def __init__(self):
        self.sum = 0
        self.count = 0
        
    def update(self, value, count):
        self.count += count
        self.sum += value*count
        
    def get_average(self):
        return self.sum/self.count
        
def train_step(model, optimizer, train_dataloader, criterion, device,scaler=None):
    avg_meter = AverageMeter()
    model.train()
    progbar = tqdm.tqdm(train_dataloader, desc='Train', total=len(train_dataloader))
    
    if scaler is None:
        for img_array, target in progbar:
            batchsize = len(target)
            img_array = img_array.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            y_pred = model(img_array)
            train_loss = criterion(y_pred.view(-1), target)
            avg_meter.update(train_loss.item(), batchsize)

            train_loss.backward()
            optimizer.step()  
            progbar.set_postfix({"train_loss":f"{avg_meter.get_average():.5f}"})
    else:
        for img_array, target in progbar:
            batchsize = len(target)
            img_array = img_array.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                y_pred = model(img_array)
                train_loss = criterion(y_pred.view(-1), target)
    
            avg_meter.update(train_loss.item(), batchsize)
            scaler.scale(train_loss).backward()
            scaler.step(optimizer)
            scaler.update()
    
            progbar.set_postfix({"train_loss":f"{avg_meter.get_average():.5f}"})

    return avg_meter.get_average()

def validate_step(model, val_dataloader, criterion, device):
    avg_meter = AverageMeter()
    model.eval()
    progbar = tqdm.tqdm(val_dataloader, desc="Validation", total=len(val_dataloader))
    for img_array, target in progbar:
        batchsize = len(target)
        img_array = img_array.to(device)
        target = target.to(device)
        with torch.no_grad():
            y_pred = model(img_array)
            val_loss = criterion(y_pred.view(-1), target)
            
        avg_meter.update(val_loss, batchsize)
        progbar.set_postfix({"val_loss":f"{avg_meter.get_average():.5f}"})
        
    return avg_meter.get_average()
    

def get_scheduler(optimizer, config):
    if Config.scheduler == "CosineAnnealingLR":
        return CosineAnnealingLR(optimizer, **config.scheduler_params)
    if Config.scheduler == "ReduceLROnPlateau":
        return ReduceLROnPlateau(optimizer, **config.scheduler_params)
    
def get_optimizer(model, config):
    if config.optimizer == "Adam":
        return torch.optim.Adam(model, **config.optimizer_params)
    if config.optimizer == "SGD":
        return torch.optim.SGD(model, **config.optimizer_params)
    
    raise NotImplementedError

def get_dataloaders(train_ds, val_ds, train_batchsize=Config.train_batchsize, val_batchsize=Config.val_batchsize):
    train_transform = A.Compose([
#         A.RandomResizedCrop(Config.image_size, Config.image_size, scale=(0.7,1.0),p=0.8),
#         A.HorizontalFlip(p=0.5),
        A.Resize(Config.image_size, Config.image_size,p=1),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
        ToTensorV2()], p=1)
    
    val_transform = A.Compose([
    A.Resize(Config.image_size, Config.image_size,p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
    ToTensorV2()], p=1)
    
    train_ds = PawDataset(train_ds["filepath"].to_numpy(), train_ds["Pawpularity"].to_numpy(), transforms=train_transform)
    val_ds = PawDataset(val_ds["filepath"].to_numpy(), val_ds["Pawpularity"].to_numpy(), transforms=val_transform)
    
    train_dataloader = DataLoader(train_ds, batch_size=train_batchsize, shuffle=True, pin_memory=True, drop_last=True)
    val_dataloader = DataLoader(val_ds, batch_size=val_batchsize, shuffle=False, pin_memory=True)
    
    return train_dataloader, val_dataloader


## Training loop

In [ ]:
for n_fold in range(Config.n_splits):
    if n_fold in [0]:
        print("Perfoming training for fold:", n_fold)
        train_ds = train_df[train_df['n_fold']!=n_fold]
        val_ds = train_df[train_df['n_fold']==n_fold]

        # preparing the data
        train_dataloader, val_dataloader = get_dataloaders(train_ds, val_ds, 
                                                          train_batchsize=Config.train_batchsize,
                                                          val_batchsize=Config.val_batchsize)

        #device
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # creating the model
        model = PawModel(Config.model_name)
        model = model.to(device)

        # get optimizer
        optimizer = get_optimizer(model.parameters(), Config)

        # get scheduler 
        scheduler = get_scheduler(optimizer, Config)

        # fp16
        scaler =None
        if Config.fp16:
            scaler = torch.cuda.amp.GradScaler()

        # loss function
        criterion = RMSELoss()

        # loss history
        train_loss_history = []
        val_loss_history = []
        best_val_loss = float("inf")

        for n_epoch in range(Config.epochs):
            print(f"EPOCH {n_epoch}/{Config.epochs}")
            avg_train_loss = train_step(model, optimizer, train_dataloader, criterion, device, scaler) 
            train_loss_history.append(avg_train_loss)

            avg_val_loss =  validate_step(model, val_dataloader, criterion, device)
            val_loss_history.append(avg_val_loss)

            if Config.scheduler == "ReduceLROnPlateau":
                scheduler.step(avg_val_loss)
            else:
                scheduler.step()

            # save model
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                model.eval()
                torch.save(model.state_dict(), os.path.join("./", f"{Config.model_name}_n_fold_{n_fold}_best.pth"))

            # Log to wandb
            wandb.log({
                f"fold_{n_fold}_epoch": n_epoch+1,
                f"fold_{n_fold}_train_loss": avg_train_loss,
                f"fold_{n_fold}_val_loss": avg_val_loss,
            })
    

In [ ]:
wandb.finish()